In [32]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [33]:
# Load dataset
df_titanic = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\titanic.csv')

In [34]:
# Sets null values equal to the median of that column
def handle_null_median(df):
    # Need to set inplace=True, so it doesn't create a copy of the dataframe. Tried without and this led to null-values not being removed
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Age'].fillna(df['Age'].median(), inplace=True)

    df['Embarked'].fillna('S', inplace=True)

    return  df

df_titanic = handle_null_median(df_titanic)
df_titanic['Sex'] = df_titanic['Sex'].replace(['female', 'male'], [0,1])
df_titanic['FamilyMembersCount'] = df_titanic['Parch'] + df_titanic['SibSp'] + 1
df_titanic = df_titanic.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp', 'Parch'], axis=1)

In [35]:
df_XGB = df_titanic.copy()
df_RF = df_titanic.copy()
df_GNB = df_titanic.copy()

In [36]:
def bins(df): # This is kind of feature engineering as well
    df['AgeGroup'] = 0
    df.loc[df['Age'] < 16.336, 'AgeGroup' ] = 0
    df.loc[(df['Age'] >= 16.336) & (df['Age'] < 32.252), 'AgeGroup'] = 1
    df.loc[(df['Age'] >= 32.252) & (df['Age'] < 48.168), 'AgeGroup'] = 2
    df.loc[(df['Age'] >= 48.168) & (df['Age'] < 64.084), 'AgeGroup'] = 3
    df.loc[df['Age'] >= 64.084, 'AgeGroup'] = 4

    # Could also create bins for fare, but not sure Fare is needed.
    return df

df_titanic = bins(df_titanic)

In [37]:
df_titanic['Pclass'] =  df_titanic['Pclass'].astype('category')
df_titanic['Embarked'] =  df_titanic['Embarked'].astype('category')
# df_titanic['AgeGroup'] =  df_titanic['AgeGroup'].astype('category')
df_titanic['Sex'] =  df_titanic['Sex'].astype('category')
df_titanic = df_titanic.drop('Age', axis=1)

# XGBoost

In [38]:
df_XGB = pd.get_dummies(df_titanic)
df_XGB = df_XGB.drop(['Fare'], axis=1)
df_XGB.head()

,Survived,FamilyMembersCount,AgeGroup,Pclass_1,Pclass_2,Pclass_3,Sex_0,Sex_1,Embarked_C,Embarked_Q,Embarked_S
0,0,2,1,0,0,1,0,1,0,0,1
1,1,2,2,1,0,0,1,0,1,0,0
2,1,1,1,0,0,1,1,0,0,0,1
3,1,2,2,1,0,0,1,0,0,0,1
4,0,1,2,0,0,1,0,1,0,0,1


In [39]:
train_X, test_X, train_y, test_y = train_test_split(df_XGB.drop('Survived', axis=1), df_XGB['Survived'], random_state=42)

XGBModel = XGBClassifier(verbosity=0)

XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

(0.8069240264841208, 0.03055357697169612)

In [40]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.8071748878923767

# XGB TUNING

In [41]:
XGBParam1 = {
    'max_depth': [2,4,6,8,10],
    'min_child_weight': [2,4,6,8]
    }

XGB_Grid1 = GridSearchCV(XGBClassifier(verbosity=0,
                                       gamma=0,
                                       subsample=0.6,
                                       learning_rate=0.1,
                                       n_estimators=200), XGBParam1)
XGB_Grid1.fit(train_X, train_y)
print(XGB_Grid1.best_params_, XGB_Grid1.best_score_)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

KeyboardInterrupt: 

In [ ]:
XGBParam2 = {
    'gamma':[0.0,0.1,0.2,0.3,0.4,0.5]
}

XGB_Grid2 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=6,
                                       min_child_weight=6,
                                       subsample=0.6,
                                       learning_rate=0.1,
                                       n_estimators=200), XGBParam2)
XGB_Grid2.fit(train_X, train_y)
print(XGB_Grid2.best_params_, XGB_Grid2.best_score_)

In [ ]:
XGBParam3 = {
    'subsample':[0.5,0.6,0.7]
}

XGB_Grid3 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=6,
                                       min_child_weight=6,
                                       gamma=0,
                                       learning_rate=0.04,
                                       n_estimators=200), XGBParam3)
XGB_Grid3.fit(train_X, train_y)
print(XGB_Grid3.best_params_, XGB_Grid3.best_score_)

In [ ]:
XGBParam4 = {
    'learning_rate':[0.03,0.04,0.06]
}

XGB_Grid4 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=6,
                                       min_child_weight=6,
                                       gamma=0,
                                       subsample=0.7,
                                       n_estimators=100), XGBParam4)
XGB_Grid4.fit(train_X, train_y)
print(XGB_Grid4.best_params_, XGB_Grid4.best_score_)

In [ ]:
XGBParam5 = {
    'n_estimators':[60, 80, 100]
}

XGB_Grid5 = GridSearchCV(XGBClassifier(verbosity=0,
                                       max_depth=6,
                                       min_child_weight=6,
                                       gamma=0,
                                       subsample=0.6,
                                       learning_rate=0.04), XGBParam5)
XGB_Grid5.fit(train_X, train_y)
print(XGB_Grid5.best_params_, XGB_Grid5.best_score_)

In [ ]:
XGBModel = XGBClassifier(verbosity=0,
                         max_depth=6,
                         min_child_weight=6,
                         gamma=0,
                         subsample=0.6,
                         learning_rate=0.04,
                         n_estimators=80)
XGB_scores = cross_val_score(XGBModel, train_X, train_y)
(XGB_scores.mean(), XGB_scores.std())

In [ ]:
XGBModel.fit(train_X, train_y)
pred = XGBModel.predict(test_X)
accuracy_score(test_y, pred)

# Random Forrest Classifier

In [ ]:
df_RF = pd.get_dummies(df_titanic)
df_RF = df_RF.drop(['Fare'], axis=1)
df_RF.head()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('Survived', axis=1), df_RF['Survived'], random_state=42)

RFModel = RandomForestClassifier()

RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

In [ ]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

In [ ]:
RFParam = {
    "max_depth":[4, 5, 6],
    "n_estimators":[50, 100, 200, 300],
    "min_samples_leaf": range(0, 10, 2),
    "max_samples": [0, .1,.2,.3]
}

RF_Grid = GridSearchCV(RandomForestClassifier(), RFParam)
RF_Grid.fit(train_X, train_y)
print(RF_Grid.best_params_, RF_Grid.best_score_)

In [ ]:
RFModel = RandomForestClassifier(max_depth=5, max_samples=0.3, min_samples_leaf=2, n_estimators=100)
RF_scores = cross_val_score(RFModel, train_X, train_y)
(RF_scores.mean(), RF_scores.std())

In [ ]:
RFModel.fit(train_X, train_y)
pred = RFModel.predict(test_X)
accuracy_score(test_y, pred)

# GaussianNB

In [ ]:
df_GNB = pd.get_dummies(df_GNB)
df_GNB = df_GNB.drop(['Fare'], axis=1)
df_GNB.head()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(df_GNB.drop('Survived', axis=1), df_RF['Survived'], random_state=42)

GNBModel = GaussianNB()

GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

In [ ]:
GNBModel.fit(train_X, train_y)
pred = GNBModel.predict(test_X)
accuracy_score(test_y, pred)

In [ ]:
GNBParam = {
    'var_smoothing': [1e-12, 1e-10, 1e-8, 1e-6, 1e-5, 1e-4, 1e-2]
}

GNB_Grid = GridSearchCV(GaussianNB(), GNBParam)
GNB_Grid.fit(train_X, train_y)
print(GNB_Grid.best_params_, GNB_Grid.best_score_)

In [ ]:
GNBModel = GaussianNB(var_smoothing=1e-6)
GNB_scores = cross_val_score(GNBModel, train_X, train_y)
(GNB_scores.mean(), GNB_scores.std())

In [ ]:
GNBModel.fit(train_X, train_y)
pred = GNBModel.predict(test_X)
accuracy_score(test_y, pred)